## About the Agent

This is an agent that autonomously generates and executes code in a secure cloud sandbox to respond to user queries. It specifically works on data to perform statistical analysis and creates visualizations.

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from helper import load_env
load_env()

# Tool Calling

- **Goal:** Create function schemas that tell the LLM what tools are available and how to call them
- **Workflow:** Need something like execute_code for running Python, or write_file for creating files
- **Info:** Tools are called by the LLM via function calling, so schemas must be precise

In [ ]:
import json
from typing import Callable
from e2b_code_interpreter import Execution, Sandbox

# Tool Schema - tells the LLM what tools are available
execute_code_schema = {
    "type": "function",
    "name": "execute_code",
    "description": "Execute Python code and return the result or error.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "Python code to execute as a string.",
            }
        },
        "required": ["code"],
        "additionalProperties": False,
    },
}

# Tool Implementation - executes code in the sandbox
def execute_code(sbx: Sandbox, code: str, language: str = "python") -> Execution:
    execution = sbx.run_code(code, language)
    metadata = {}
    results = execution.results
    for result in results:
        if result.png:
            metadata["images"] = [result.png]
            result.png = None
            result.chart = None
    return execution.to_json(), metadata

# Helper to execute any tool by name
def execute_tool(name: str, args: str, tools: dict[str, Callable], **kwargs):
    metadata = {}
    try:
        args = json.loads(args)
        if name not in tools:
            return {"error": f"Tool {name} doesn't exist."}
        result, metadata = tools[name](**args, **kwargs)
    except json.JSONDecodeError as e:
        result = {"error": f"{name} failed to parse arguments: {str(e)}"}
    except KeyError as e:
        result = {"error": f"Missing key in arguments: {str(e)}"}
    except Exception as e:
        result = {"error": str(e)}
    return result, metadata

# Agent Loop

This section is the iterative loop that powers the coding agent. This is where the LLM reasons, calls tools, receives results, and decides what to do next.

- **Goal:** Implement a multi-step agent that iterates until task completion or max steps
- **Workflow:** Create a loop that alternates between LLM calls and tool execution
- **Info:** Stop when LLM doesn't call any more functions and use max_steps to prevent infinite loops

In [ ]:
import json
import re
import logging
import base64
from openai import OpenAI
from typing import Generator, Literal, Optional, Callable
from e2b_code_interpreter import Sandbox
from IPython.display import Image, display
from rich.logging import RichHandler
from rich.panel import Panel
from rich.console import Console

# Logger setup
LOG_LEVELS = {
    "DEBUG": "🐛", "INFO": "✨", "WARNING": "⚠️",
    "ERROR": "❌", "CRITICAL": "🔥", "TOOL": "🤖",
}

logger = logging.getLogger("dlai")
logger.setLevel(logging.INFO)
logger.propagate = False
if not logger.handlers:
    handler = RichHandler(rich_tracebacks=True, show_path=False, show_time=False, show_level=True, markup=False)
    logger.addHandler(handler)

console = Console()

def log_tool_call(tool_name, args_str, result):
    """Logs a tool call with a card-like display."""
    MAX_DISPLAY_LENGTH = 200
    result_str = str(result)
    if len(args_str) > MAX_DISPLAY_LENGTH:
        args_str = args_str[:MAX_DISPLAY_LENGTH] + "..."
    if len(result_str) > MAX_DISPLAY_LENGTH:
        result_str = result_str[:MAX_DISPLAY_LENGTH] + "..."
    panel_content = f"[bold]{tool_name}[/bold]\n"
    panel_content += f"Arguments: [cyan]{args_str}[/cyan]\n"
    panel_content += f"Result: [magenta]{result_str}[/magenta]"
    console.print(Panel(panel_content, title=f"{LOG_LEVELS['TOOL']} Tool Call", expand=False))

# Compression prompt for long conversations
SYSTEM_PROMPT_COMPRESS_MESSAGES = r"""You are the component that summarizes internal chat history into a given structure.
When the conversation history grows too large, you will be invoked to distill the entire history into a concise, structured XML snapshot.

<state_snapshot>
    <overall_goal><!-- User's high-level objective --></overall_goal>
    <key_knowledge><!-- Crucial facts and constraints --></key_knowledge>
    <file_system_state><!-- Files created/modified/read --></file_system_state>
    <recent_actions><!-- Last significant agent actions --></recent_actions>
    <current_plan><!-- Step-by-step plan with status --></current_plan>
</state_snapshot>"""

TOKEN_LIMIT = 60_000
COMPRESS_THRESHOLD = 0.7
STATE_SNAPSHOT_PATTERN = re.compile(r"<state_snapshot>(.*?)</state_snapshot>", re.DOTALL)

def clean_messages_for_llm(messages: list[dict]) -> list[dict]:
    return [{k: v for k, v in msg.items() if not k.startswith("_")} for msg in messages]

def compress_messages(client: OpenAI, messages: list[dict]) -> list[dict]:
    response = client.responses.create(
        model="gpt-5-nano",
        input=[
            {"role": "developer", "content": SYSTEM_PROMPT_COMPRESS_MESSAGES},
            *messages,
            {"role": "user", "content": "First, reason in your scratchpad. Then, generate the <state_snapshot>."},
        ],
    )
    text = response.output_text
    context = "\n".join(STATE_SNAPSHOT_PATTERN.findall(text))
    return [
        {"role": "user", "content": f"This is snapshot of the conversation so far:\n{context}"},
        {"role": "assistant", "content": "Got it. Thanks for the additional context!"},
    ]

def get_compress_message_index(messages: list[dict]) -> int:
    chars = [len(json.dumps(message)) for message in messages]
    total_chars = sum(chars)
    target_chars = total_chars * COMPRESS_THRESHOLD
    curr_chars = 0
    for index, char in enumerate(chars):
        curr_chars += char
        if curr_chars >= target_chars:
            return index
    return len(messages)

def get_first_user_message_index(messages: list[dict]) -> int:
    for index, message in enumerate(messages):
        if "role" in message and message["role"] == "user":
            return index
    return 0

def maybe_compress_messages(client: OpenAI, messages: list[dict], usage: int) -> list[dict]:
    if usage <= TOKEN_LIMIT * COMPRESS_THRESHOLD:
        return messages
    compress_index = get_compress_message_index(messages)
    if compress_index >= len(messages):
        return messages
    compress_index += get_first_user_message_index(messages[compress_index:])
    if compress_index <= 0:
        return messages
    last_message = messages[compress_index - 1]
    if "type" in last_message and last_message["type"] == "function_call":
        compress_index += 1
    to_compress_messages = messages[:compress_index]
    to_keep_messages = messages[compress_index:]
    if len(to_compress_messages) > 0:
        logger.info(f"[agent] compressing messages [0...{compress_index}]...")
        return [*compress_messages(client, to_compress_messages), *to_keep_messages]
    return messages

def coding_agent(
    client: OpenAI,
    sbx: Sandbox,
    query: str,
    tools: dict[str, Callable],
    tools_schemas: list[dict],
    max_steps: int = 5,
    system: Optional[str] = "You are a senior python programmer",
    messages: Optional[list[dict]] = None,
    usage: Optional[int] = 0,
    model: Literal["gpt-4.1-mini", "gpt-5-mini"] = "gpt-4.1-mini",
    **model_kwargs,
) -> Generator[tuple[dict, dict, int], None, tuple[list[dict], int]]:
    """Main agent loop that iterates between LLM calls and tool execution."""
    if messages is None:
        messages = []
    
    user_message = {"role": "user", "content": query}
    messages.append(user_message)
    yield user_message, messages, usage

    steps = 0
    while steps < max_steps:
        messages = maybe_compress_messages(client, clean_messages_for_llm(messages), usage)
        response = client.responses.create(
            model=model,
            input=[
                {"role": "developer", "content": system},
                *clean_messages_for_llm(messages),
            ],
            tools=tools_schemas,
            **model_kwargs,
        )
        usage = response.usage.total_tokens
        has_function_call = False
        
        for part in response.output:
            messages.append(part.to_dict())
            yield part.to_dict(), messages, usage
            if part.type == "function_call":
                has_function_call = True
                name = part.name
                result, metadata = execute_tool(name, part.arguments, tools, sbx=sbx)
                result_msg = {
                    "type": "function_call_output",
                    "call_id": part.call_id,
                    "output": json.dumps(result),
                    "_metadata": metadata,
                }
                messages.append(result_msg)
                yield result_msg, messages, usage

        steps += 1
        if not has_function_call:
            break

    return messages, usage

def log(generator_func, *args, **kwargs):
    """Wraps the coding_agent and handles logging."""
    gen = generator_func(*args, **kwargs)
    step = 0
    pending_tool_calls = {}

    try:
        while True:
            part_dict, messages, usage = next(gen)
            part_type = part_dict.get("type")

            if part_type == "reasoning":
                if step == 0:
                    logger.info(f"[agent-#{step}] Thinking...")
                    step += 1
                logger.info(" ...")
            elif part_type == "message":
                content = part_dict.get("content")
                if content and content[0].get("text"):
                    logger.info(f"{content[0]['text']}")
            elif part_type == "function_call":
                call_id = part_dict.get("call_id")
                name = part_dict.get("name")
                arguments = part_dict.get("arguments")
                pending_tool_calls[call_id] = (name, arguments)
            elif part_type == "function_call_output":
                call_id = part_dict.get("call_id")
                if call_id in pending_tool_calls:
                    name, arguments = pending_tool_calls.pop(call_id)
                    result = json.loads(part_dict.get("output", "{}"))
                    log_tool_call(name, arguments, result)
                metadata = part_dict.get("_metadata")
                if metadata:
                    images = metadata.get("images")
                    if images:
                        for image in images:
                            display(Image(data=base64.b64decode(image)))

    except StopIteration as e:
        messages, final_usage = e.value
        logger.info(f"[agent] tokens: {final_usage} total")
        return messages, final_usage

# Sandbox Execution

Use the agent in the cloud through a sandbox. Instead of running code locally, execute everything in an E2B sandbox which is a secure, isolated environment for untrusted code.

- **Goal:** Integrate E2B sandbox with the coding agent for safe cloud execution
- **Workflow:** Create sandbox, modify execute_code to use sandbox, update agent to pass sandbox to tools
- **Remember:** Sandboxes are persistent, so you can reconnect, query by metadata, and serve websites from them

In [ ]:
from e2b_code_interpreter import Sandbox, SandboxQuery, SandboxState
from uuid import uuid4
from pathlib import Path

CACHE_FILENAME = "sbx.cache"

def create_sandbox(template: str = None, overwrite: bool = False, **kwargs) -> Sandbox:
    """Create or reconnect to a sandbox."""
    cache_file = Path(CACHE_FILENAME)

    if cache_file.exists():
        name = cache_file.read_text()
    else:
        name = f"dlai-sbx-{template}-{uuid4()}"
        cache_file.write_text(name)

    if not overwrite:
        running_sandboxes = Sandbox.list(
            SandboxQuery(metadata={"name": name}, state=[SandboxState.RUNNING])
        ).next_items()
        if running_sandboxes:
            sandbox = Sandbox.connect(running_sandboxes[0].sandbox_id)
            logger.info(f"[sandbox] Reconnecting to Sandbox(id={sandbox.sandbox_id})")
            return sandbox

    sandbox = Sandbox.create(timeout=60 * 60, metadata={"name": name}, template=template, **kwargs)
    logger.info(f"[sandbox] Creating new Sandbox(id={sandbox.sandbox_id})")
    return sandbox

# Create the sandbox
sbx = create_sandbox()

# Upload the data file to the sandbox
with open("pokemon.csv", "r") as f:
    content = f.read()

sbx.files.write("data.csv", content)

In [ ]:
from openai import OpenAI

client = OpenAI()

system = """You are a senior python programmer. 
You must run the code using the `execute_code` tool.
The user has uploaded a data.csv.
You help the user understanding the data 
by creating interesting plots.
"""

# Map tool names to implementations
tools = {"execute_code": execute_code}

In [ ]:
messages = []

query = "What is the data about?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

In [ ]:
query = "Can you aggregate the pokemons by type?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

# Gradio UI

Below is the Gradio UI to have a nicer user experience chatting with the code generation agent. It has some built in features to provide information such as context stack, number of tokens used in the result, etc.

In [ ]:
from lib.ui import ui

ui(coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

**Resources:**
- [E2B Documentation](https://e2b.dev/docs)
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [E2B GitHub](https://github.com/e2b-dev/e2b)